In [1]:
%run 1-workspace_setup.ipynb

In [2]:
dem_norm = read_main('county_stats_normalized')
dem_cnt = read_main('county_stats_counts')

edu_norm = read_main('education_county_rates')
edu_cnt = read_main('education_county_counts')

cr_base_rate = read_main('crime_agg_base_rate')
cr_base_count = read_main('crime_agg_base_count')
cr_cat = read_main('crime_agg_category')
cr_name = read_main('crime_agg_name')
cr_cat_rate = read_main('crime_agg_category_rate')
cr_name_rate = read_main('crime_agg_name_rate')

# head(dem_norm, dem_cnt, edu_norm, edu_cnt, cr_base_count, cr_base_rate, cr_cat, cr_name, cr_cat_rate, cr_name_rate)

In [3]:
INDEX = ['year', 'county']

def non_idx_cols(df, index=INDEX):
    return [col for col in df.columns if col not in index]

In [4]:
cr_name = cr_name.prefix_cols(non_idx_cols(cr_name), 'CRIME_NAME_COUNT__')
cr_cat = cr_cat.prefix_cols(non_idx_cols(cr_cat), 'CRIME_CATEGORY_COUNT__')

cr_name_rate = cr_name_rate.prefix_cols(non_idx_cols(cr_name_rate), 'CRIME_NAME_NORM__')
cr_cat_rate = cr_cat_rate.prefix_cols(non_idx_cols(cr_cat_rate), 'CRIME_CATEGORY_NORM__')

cr_base_count = cr_base_count.prefix_cols(non_idx_cols(cr_base_count), 'CRIME_BASE_COUNT__')
cr_base_rate = cr_base_rate.prefix_cols(non_idx_cols(cr_base_rate), 'CRIME_BASE_NORM__')

dem_cnt = dem_cnt.prefix_cols(non_idx_cols(dem_cnt), 'CENSUS_COUNT__')
dem_norm = dem_norm.prefix_cols(non_idx_cols(dem_norm), 'CENSUS_NORM__')

edu_cnt = edu_cnt.prefix_cols(non_idx_cols(edu_cnt, ['county']), 'EDU_COUNT__')
edu_norm = edu_norm.prefix_cols(non_idx_cols(edu_norm, ['county']), 'EDU_NORM__')

# head(cr_name, cr_cat, cr_name_rate, cr_cat_rate, cr_base_count, cr_base_rate, dem_cnt, dem_norm, edu_cnt, edu_norm)

In [5]:
df = (
    # CRIME
    cr_name
    .merge(cr_cat, on=INDEX)
    .merge(
        (
            cr_name_rate
            .merge(cr_cat_rate, on=INDEX)
        ),
        on=INDEX
    )
    .merge(
        (
            cr_base_count
            .merge(cr_base_rate, on=INDEX)
        ),
        on=INDEX
    )
    # DEMOGRAPHICS
    .merge(
        (
            dem_cnt
            .merge(dem_norm, on=INDEX)
        ),
        on=INDEX
    )
    # EDUCATION
    .merge(
        (
            edu_cnt
            .merge(edu_norm, on=['county'])
        ),
        on=['county']
    )
)

In [6]:
# Change all numbers to floats, and round
numeric_cols = df.select_dtypes(include=[np.number])
df[numeric_cols.columns] = numeric_cols.astype(float).round(2)
df.year = df.year.astype(int)
head(df)

381 cols x 490 rows


,year,county,CRIME_NAME_COUNT__arson,CRIME_NAME_COUNT__assault_aggravated,CRIME_NAME_COUNT__assault_simple,CRIME_NAME_COUNT__bribery,CRIME_NAME_COUNT__burglary,CRIME_NAME_COUNT__credit_card_machine_fraud,CRIME_NAME_COUNT__drug_equipment,CRIME_NAME_COUNT__drug_narcotic,...,EDU_NORM__white_mobile_instances_rate,EDU_NORM__black_stable_rate,EDU_NORM__black_mobile_rate,EDU_NORM__black_mobile_instances_rate,EDU_NORM__hispanic_stable_rate,EDU_NORM__hispanic_mobile_rate,EDU_NORM__hispanic_mobile_instances_rate,EDU_NORM__asian_stable_rate,EDU_NORM__asian_mobile_rate,EDU_NORM__asian_mobile_instances_rate
0,2012,ADAMS,105.0,1029.0,2835.0,2.0,2311.0,535.0,3211.0,3522.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
1,2013,ADAMS,73.0,899.0,2649.0,7.0,2144.0,437.0,1965.0,2204.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
2,2014,ADAMS,123.0,1033.0,2863.0,8.0,2073.0,526.0,1945.0,2145.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54


In [7]:
## TEMPORARY - REVERSE OUR WORK
df.columns = [c if '__' not in c else c.split('__')[1] for c in df.columns]
df = df.loc[:, ~ df.columns.duplicated()]
head(df)

353 cols x 490 rows


,year,county,arson,assault_aggravated,assault_simple,bribery,burglary,credit_card_machine_fraud,drug_equipment,drug_narcotic,...,white_mobile_instances_rate,black_stable_rate,black_mobile_rate,black_mobile_instances_rate,hispanic_stable_rate,hispanic_mobile_rate,hispanic_mobile_instances_rate,asian_stable_rate,asian_mobile_rate,asian_mobile_instances_rate
0,2012,ADAMS,105.0,1029.0,2835.0,2.0,2311.0,535.0,3211.0,3522.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
1,2013,ADAMS,73.0,899.0,2649.0,7.0,2144.0,437.0,1965.0,2204.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54
2,2014,ADAMS,123.0,1033.0,2863.0,8.0,2073.0,526.0,1945.0,2145.0,...,32.45,54.66,45.08,47.91,67.49,32.71,36.4,78.55,21.37,23.54


In [8]:
write_main(df, 'county_EVERYTHING')

490

## District

In [9]:
df_dist = read_main('education_dist')
df_geo_dist = read_main('''
SELECT
    d.district,
    c.county,
    d.geo_border AS geo_district_border,
    c.geo_border AS geo_county_border,
    d.geo_point AS geo_district_point,
    c.geo_point AS geo_county_point
FROM district AS d
INNER JOIN county AS c
    ON c.county = d.in_county
''')

In [10]:
df_dist = (df_geo_dist
    .merge(df_dist, on=['district'])
    [ ~ df_geo_dist.geo_district_border.isna()]
)

head(df_dist)

142 cols x 178 rows


,district,county,geo_district_border,geo_county_border,geo_district_point,geo_county_point,pupil_total,stable,mobile,mobile_instances,...,white_mobile_instances_rate,black_stable_rate,black_mobile_rate,black_mobile_instances_rate,hispanic_stable_rate,hispanic_mobile_rate,hispanic_mobile_instances_rate,asian_stable_rate,asian_mobile_rate,asian_mobile_instances_rate
0,MAPLETON 1,ADAMS,MULTIPOLYGON (((-105.01581612299998 39.8144774...,MULTIPOLYGON (((-103.70574149517748 39.9999110...,GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,9037,5077,3919,4133,...,52.08,48.04,51.40,53.63,60.31,39.19,42.19,47.22,52.78,53.70
1,ADAMSFIVESTAR 12,ADAMS,MULTIPOLYGON (((-105.05310614499996 39.9302934...,MULTIPOLYGON (((-103.70574149517748 39.9999110...,GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,49889,34283,15424,16854,...,32.27,55.98,43.94,47.41,67.30,32.23,36.81,81.07,18.84,21.15
2,ADAMSCOUNTY 14,ADAMS,MULTIPOLYGON (((-104.96883410999999 39.7910064...,MULTIPOLYGON (((-103.70574149517748 39.9999110...,GEOMETRYCOLLECTION EMPTY,GEOMETRYCOLLECTION EMPTY,8265,5510,3038,3397,...,47.45,48.20,51.80,53.60,68.67,35.20,39.57,80.00,20.00,20.00


In [11]:
write_main(df_dist, 'district_EVERYTHING')

178